In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random
from collections import OrderedDict
import math

In [2]:
#Hadamard matrices

H = {}
H[4] = torch.tensor([[1.0, 1.0, 1.0, 1.0], [1.0, -1.0, 1.0, -1.0], [1.0, 1.0, -1.0, -1.0], [1.0, -1.0, -1.0, 1.0]])
H[8] = torch.tensor([[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
                            [1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0],
                            [1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0],
                            [1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0],
                            [1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0],
                            [1.0, -1.0, 1.0, -1.0, -1.0, 1.0, -1.0, 1.0],
                            [1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0],
                            [1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, -1.0]])


f = open("had64.txt")
H64 = []
for i in range(64):
    H64.append([1.0 if i=='+' else -1.0 for i in list(f.readline())[:-1]])
    
H[64] = H64

In [6]:
#Generating boolean functions and from them, their walsh spectra to feed as inputs to the neural network

#k = number of variables in the boolean functions, n = number of inputs to train the model on
def generate_input_data(k, n, scheme='regular'):
    two_pow_k, data = pow(2, k), []
    if(scheme=='regular'):
        if(n<two_pow_k):
            print("Requested number of inputs less than 2^k. Please request a higher number")
            return data
        while(True):
            if(two_pow_k<20):
                perm = torch.randperm(pow(2, two_pow_k))
                data = [[0.0]*(two_pow_k-len(bin(num)[2:]))+[float(i) for i in bin(num)[2:]] for num in perm[:n]]
            else:
                data = [[0.0 if random.random()>0.5 else 1.0 for i in range(two_pow_k)] for i in range(n)]
            rank = np.linalg.matrix_rank(data)
            if(rank<two_pow_k):
                print("Rank (", rank, ") not large enough, generating data again")
            else:
                data = torch.matmul(torch.tensor(data), H[two_pow_k])
                print("Data generated.")
                break
    elif(scheme=='one-hot'):
        data = [[0.0]*i + [1.0] + [0.0]*(two_pow_k-i-1) for i in range(two_pow_k)]
        data = torch.matmul(torch.tensor(data), H[two_pow_k])
        print("Data generated")
    return data

k, n = 3, pow(2, pow(2, k))
walsh_spectra = generate_input_data(k, n)

Data generated.


In [18]:
#Calculating the correlation immunity of functions given their walsh spectra

def correlation_immunity(walsh_spectra):
    ci = []
    n, two_pow_k = walsh_spectra.size()[0], walsh_spectra.size()[1]
    k = int(math.log2(two_pow_k))
    no_ones = []
    for i in range(two_pow_k):
        no_ones.append(sum([int(dig) for dig in bin(i)[2:]]))
    for spectrum in walsh_spectra:
        m_ci = [1]*(k+1)
        for i in range(two_pow_k):
            if(spectrum[i]!=0):
                m_ci[no_ones[i]] = 0
        m = 1
        while(m<k+1 and m_ci[m]==1):
            m+=1
        m -= 1
        # Let ci item = [x0, x1, ..., xm]. x0 is 1 if it is balanced, xi is 1 if it is i-correlation-immune
        ci.append(([1] if spectrum[0]==two_pow_k//2 else [0])+(m)*[1]+(k-m)*[0])
    return ci

print(correlation_immunity(walsh_spectra))

[[0, 0, 0, 0], [0, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 0], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [1, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 0], [1, 1, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 1, 0, 0], [0, 1, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0

In [ ]:
class NetworkModel(nn.Module):

    def __init__(self, k):

        # Initialize the network layers.

        super(NetworkModel, self).__init__()
        two_pow_k = pow(2, k)
        self.lin1 = nn.Linear(two_pow_k, two_pow_k, bias=True)

    def forward(self, x):

        # A forward function
        # Linear function without activation

        x = self.lin1(x)
        
#         F-> WalshSpec
#         WalshSpec = F * W_t

        return x


In [16]:
# print(len([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0]))
# print(walsh_spectra[240])
# print(correlation_immunity(walsh_spectra)[240])

# two_pow_k, n = 16, 12
# data = torch.tensor([[0.0 if random.random()>0.5 else 1.0 for i in range(two_pow_k)] for i in range(n)])
# print(data)

# print(walsh_spectra[:10])
# print(k)

# two_pow_k = 8
# no_ones = []
# for i in range(two_pow_k):
#     no_ones.append(sum([int(dig) for dig in bin(i)[2:]]))
# print(no_ones)

tensor([ 4.,  0.,  0.,  0.,  0.,  0.,  0., -4.])
[1, 1, 0]
